# PostgreSQL Integration Verification

This notebook tests the entire backend flow with the new PostgreSQL + SQLAlchemy architecture.

In [9]:
import requests
import uuid
import json
import time

BASE_URL_AUTH = "http://localhost:8000/api/v1/auth"
BASE_URL_ADMIN = "http://localhost:8000/api/v1/admin/interviews"
BASE_URL_CANDIDATE = "http://localhost:8000/api/v1/candidate/interviews"
BASE_URL_SESSION = "http://localhost:8000/api/v1/session"
BASE_URL_SUBMIT = "http://localhost:8000/api/v1/submit/submit"


## Test 1: Register Admin

Register a new admin with profile information.

In [10]:
# Generate unique username
unique_id = str(uuid.uuid4())[:8]
admin_user = f"admin_{unique_id}"
admin_email = f"{admin_user}@example.com"
password = "admin"

admin_payload = {
    "username": admin_user,
    "email": admin_email,
    "password": password,
    "role": "admin",
    "profile": {
        "first_name": "HR",
        "last_name": "Manager",
        "department": "Data Science Hiring",
        "designation": "Lead Recruiter"
    }
}

print(f"Registering admin: {admin_user}")
resp = requests.post(f"{BASE_URL_AUTH}/register/admin", json=admin_payload)
print(f"Status Code: {resp.status_code}")
if resp.status_code == 201:
    print("✓ Admin registered successfully")
else:
    print(f"✗ Registration failed: {resp.text}")


Registering admin: admin_193d9a7e
Status Code: 201
✓ Admin registered successfully


## Test 2: Login Admin

Login with the registered admin credentials to retrieve a JWT.

In [11]:
login_payload = {
    "username": admin_user,
    "password": "admin"
}

resp = requests.post(f"{BASE_URL_AUTH}/login/admin", json=login_payload)
print(f"Status Code: {resp.status_code}")
if resp.status_code == 200:
    print("✓ Admin login successful")
    admin_token = resp.json()["access_token"]
    admin_headers = {"Authorization": f"Bearer {admin_token}"}
else:
    print(f"✗ Login failed: {resp.text}")


Status Code: 200
✓ Admin login successful


## Test 3: Admin Candidate Registration

Admin creates a candidate. The direct unauthenticated `/register/candidate` endpoint returns 501 natively now.

In [12]:
new_cand_email = f"cand_{str(uuid.uuid4())[:6]}@test.com"
files = {
    "resume": ("resume.pdf", b"fake_content", "application/pdf")
}
data = {
    "candidate_name": "New Candidate",
    "candidate_email": new_cand_email,
    "job_description": "Senior Python Dev"
}

resp = requests.post(f"{BASE_URL_AUTH}/admin/register-candidate", headers=admin_headers, files=files, data=data)
print(f"Status Code: {resp.status_code}")
if resp.status_code == 201:
    print("✓ Candidate registered by Admin successfully")
    candidate_id = resp.json()["id"]
    candidate_user = resp.json()["username"]
else:
    print(f"✗ Admin registration failed: {resp.text}")


Status Code: 201
✓ Candidate registered by Admin successfully


## Test 4: Admin Interview Scheduling

Admin lists templates, picks one, and schedules the candidate.

In [13]:
# Get active templates
templates_resp = requests.get(f"{BASE_URL_ADMIN}/templates", headers=admin_headers)
if templates_resp.status_code == 200 and templates_resp.json():
    print("✓ Templates fetched successfully")
    template_id = templates_resp.json()[0]["id"]
else:
    print("✗ Failed to fetch templates or none active (Did you run seed_db.py?)")

import datetime
future_date = (datetime.datetime.utcnow() + datetime.timedelta(days=2)).isoformat() + "Z"

schedule_payload = {
    "candidate_id": candidate_id,
    "template_id": template_id,
    "scheduled_at": future_date
}

resp = requests.post(f"{BASE_URL_ADMIN}/schedule", headers=admin_headers, json=schedule_payload)
print(f"Status Code: {resp.status_code}")
if resp.status_code == 201:
    print("✓ Interview Scheduled successfully")
    interview_id = resp.json()["id"]
else:
    print(f"✗ Scheduling failed: {resp.text}")


✗ Failed to fetch templates or none active (Did you run seed_db.py?)


C:\Users\absaa\AppData\Local\Temp\ipykernel_7916\1308907151.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  future_date = (datetime.datetime.utcnow() + datetime.timedelta(days=2)).isoformat() + "Z"


NameError: name 'template_id' is not defined

## Test 5: Verify Admin Summary

Check that the interview appears in the global admin summary.

In [14]:
summ_resp = requests.get(f"{BASE_URL_ADMIN}/summary", headers=admin_headers)
print(f"Status Code: {summ_resp.status_code}")
if summ_resp.status_code == 200:
    summary = summ_resp.json()
    found = any(s["interview_id"] == interview_id for s in summary)
    if found:
        print("✓ Interview found in admin summary")
    else:
        print("✗ Interview missing from admin summary")
else:
    print(f"✗ Failed to fetch summary: {summ_resp.text}")


Status Code: 200
✗ Interview missing from admin summary


## Test 6: Toggle Candidate Login Restrictions

Admin explicitly bans and unbans the candidate from logging in.

In [ ]:
resp = requests.post(f"{BASE_URL_AUTH}/admin/candidates/{candidate_id}/toggle-login", headers=admin_headers)
print(f"Status Code: {resp.status_code} - Disabled")

resp = requests.post(f"{BASE_URL_AUTH}/admin/candidates/{candidate_id}/toggle-login", headers=admin_headers)
print(f"Status Code: {resp.status_code} - Re-Enabled")
if resp.status_code == 200:
    print("✓ Candidate login capabilities toggled successfully")
else:
    print(f"✗ Toggle failed: {resp.text}")


## Test 7: Candidate Login Verification Mode

Since candidate passwords are auto-generated dynamically, we omit direct login flow via REST here without parsing DB externally since `test_interview_flow.py` handles DB introspection via pytest hooks natively.